In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request, json
import re
import matplotlib as plt
import sqlalchemy
import sqlite3

## WATCH OUT, WE'RE LOADING DATA HERE

In [48]:
url_endpoint = 'https://data.cityofchicago.org/resource/cwig-ma7x.json'
response = requests.get(url_endpoint)
data = response.json()
df = pd.DataFrame(data)
df.violations = df.violations.fillna('')


In [51]:
df

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,longitude,results,risk,state,violations,zip,infracs,infracs_minor,infracs_serious,infracs_critical
0,24,22616,46,564,76,5308-5312 N CLARK ST,OCTAVIO CANTINA & KITCHEN,CHICAGO,OCTAVIO CANTINA & KITCHEN,Restaurant,...,-87.66848426596569,Fail,Risk 1 (High),IL,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",60640,5,5,0,0
1,24,22616,46,564,76,5308-5312 N CLARK ST,OCTAVIO CANTINA & KITCHEN,CHICAGO,OCTAVIO CANTINA & KITCHEN,Restaurant,...,-87.66848426596569,Fail,Risk 1 (High),IL,,60640,0,0,0,0
2,50,21869,20,693,11,5527-5531 N Milwaukee AVE,CAFE MARBELLA,CHICAGO,CAFE MARBELLA,Restaurant,...,-87.77365984476519,Pass,Risk 1 (High),IL,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,60630,2,2,0,0
3,27,4450,42,31,20,2727 W Devon AVE,AFGHAN MARKET,CHICAGO,AFGHAN MARKET,Grocery Store,...,-87.69828625157092,Pass,Risk 2 (Medium),IL,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",60659,8,6,2,0
4,27,4450,42,320,20,2326 W Devon AVE,NEW CHICAGO PAK SWEETS,CHICAGO,"NEW CHICAGO PAK SWEETS, INC",Restaurant,...,-87.68860508498791,Fail,Risk 1 (High),IL,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,60659,11,8,1,2
5,43,21861,30,236,45,10650 S Eberhart (500E),Cullen,CHICAGO,Cullen,School,...,-87.611281758793,Fail,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60628,3,3,0,0
6,34,21190,51,798,68,2221 N LINCOLN AVE,7 PLATES CAFE,CHICAGO,7 PLATES CAFE,Restaurant,...,-87.64439904705058,Fail,Risk 1 (High),IL,19. OUTSIDE GARBAGE WASTE GREASE AND STORAGE A...,60614,10,8,1,1
7,22,21861,45,486,45,11413 S Eggleston (400W),"Haley, Alex Mod.",CHICAGO,"Haley, Alex Mod.",School,...,-87.63359285406624,Fail,Risk 1 (High),IL,"14. PREVIOUS SERIOUS VIOLATION CORRECTED, 7-42...",60628,7,6,1,0
8,1,21569,36,291,32,3233 W CERMAK RD,TAQUERIA EL PATRON,CHICAGO,EL PATRON II,Restaurant,...,-87.7064081112397,Pass,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60623,6,6,0,0
9,22,21861,18,593,72,9801 S HALSTED ST,FUSHING CO,CHICAGO,FUSHING CO,Restaurant,...,-87.64291719936475,Pass,Risk 1 (High),IL,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",60628,5,5,0,0


**Create a list of infractions for each Inspection**

In [50]:
i=0
l=[]
crt = []
ser = []
mnr = []
while i < len(df):
    s = df['violations'][i]
    t1 = re.findall("\d+",str(re.findall("\|\s\d+",s)))
    ti = len([int(i) for i in t1])
    
    #Find number of infrac type
    m = len([int(i) for i in t1 if int(i) > 29 and int(i) <71])
    s = len([int(i) for i in t1 if int(i) > 14 and int(i) <30])
    c = len([int(i) for i in t1 if int(i) > 0 and int(i) <16])
    
    #Create list of vals
    l.append(ti)
    mnr.append(m)
    ser.append(s)
    crt.append(c)
    i=i+1

df['infracs']=l
df['infracs_minor'] = mnr
df['infracs_serious']= ser
df['infracs_critical']= crt

In [38]:
#TEST CELL
s = df['violations'][8]
t1 = re.findall("\d+",str(re.findall("\|\s\d+",s)))
m = len([int(i) for i in t1 if int(i) > 29 and int(i) <71])
t2

2

In [46]:
critical = range(1,15)
serious = range(15,30)
minor = range(30,70)

## DATABASE PARTY ZONE

In [52]:
#Prep DataFrame for SQLite
l = range(8,len(df.columns))
testdf = df.iloc[:,l]
testdf = testdf.drop(['location','violations'], axis =1)
testdf.head()

,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,longitude,results,risk,state,zip,infracs,infracs_minor,infracs_serious,infracs_critical
0,OCTAVIO CANTINA & KITCHEN,Restaurant,2018-02-22T00:00:00.000,2145344,License,41.978342059241015,2542744,-87.66848426596569,Fail,Risk 1 (High),IL,60640,5,5,0,0
1,OCTAVIO CANTINA & KITCHEN,Restaurant,2018-02-22T00:00:00.000,2145352,License,41.978342059241015,2542745,-87.66848426596569,Fail,Risk 1 (High),IL,60640,0,0,0,0
2,CAFE MARBELLA,Restaurant,2018-02-22T00:00:00.000,2145301,Canvass,41.982350141069084,2385788,-87.77365984476519,Pass,Risk 1 (High),IL,60630,2,2,0,0
3,AFGHAN MARKET,Grocery Store,2018-02-22T00:00:00.000,2145302,License Re-Inspection,41.99747344436516,2578834,-87.69828625157092,Pass,Risk 2 (Medium),IL,60659,8,6,2,0
4,"NEW CHICAGO PAK SWEETS, INC",Restaurant,2018-02-22T00:00:00.000,2145336,Canvass,41.997861005185875,2522241,-87.68860508498791,Fail,Risk 1 (High),IL,60659,11,8,1,2


In [ ]:
test.df

In [83]:
testdf.columns

Index(['address', 'aka_name', 'city', 'dba_name', 'facility_type',
       'inspection_date', 'inspection_id', 'inspection_type', 'latitude',
       'license_', 'location', 'longitude', 'results', 'risk', 'state',
       'violations', 'zip', 'infracs'],
      dtype='object')

In [66]:
#Create DB, load Panda DF
conn = sqlite3.connect("FoodInspect.db")
c = conn.cursor()
testdf.to_sql("Inspections",conn, if_exists="replace")
t = pd.read_sql_query("select * from Inspections limit 5;", conn)
conn.close()

In [112]:
t.columns

Index(['index', 'address', 'aka_name', 'city', 'dba_name', 'facility_type',
       'inspection_date', 'inspection_id', 'inspection_type', 'latitude',
       'license_', 'location', 'longitude', 'results', 'risk', 'state',
       'violations', 'zip', 'infracs'],
      dtype='object')

In [111]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

MSIA_SETTINGS = ".\config.py"
app = Flask(__name__)
app.config.from_envvar(MSIA_SETTINGS, silent=True)
db = SQLAlchemy(app)

/anaconda/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:774: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
/anaconda/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [110]:
import sqlalchemy as sa
config = {'db.url':'sqlite:////Users/matthewgallagher/MSiA/Winter_2018/MSIA_423/FoodInspection/develop/FoodInspect.db', 'db.echo':'True'}
engine = sa.engine_from_config(config, prefix='db.')
testdf.to_sql("Inspections",engine,if_exists="replace")

2018-02-20 14:59:32,504 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-02-20 14:59:32,506 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,510 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-02-20 14:59:32,511 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,514 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Inspections")
2018-02-20 14:59:32,515 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,520 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Inspections")
2018-02-20 14:59:32,521 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,524 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-02-20 14:59:32,525 INFO sqlalchemy.engine.base.Engine ()
2018-02-20 14:59:32,528 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Inspections")
2018-02-20 14:59:32,530 INFO sqlalchemy.engine.base.E

2018-02-20 14:59:32,795 INFO sqlalchemy.engine.base.Engine COMMIT
2018-02-20 14:59:32,800 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-02-20 14:59:32,801 INFO sqlalchemy.engine.base.Engine ()


## First stab at a model

In [6]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [63]:
Y = testdf['results']
X = testdf.drop(['results'],axis=1)


In [64]:
clf = linear_model.LogisticRegression()
clf.fit(X, Y)

ValueError: could not convert string to float: 'IL'